# Using QUBO to solve Variational Quantum Eigensolver for the Quadratic Assignment Problem

In [7]:
#INITIALISATION CELL
from qiskit import BasicAer,Aer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver,VQE
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToIsing

import pandas as pd
import numpy as np

The functions below are for inputting the csv data (that is in the QAPLIB format) and making the matrices numpy arrays.

In [8]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [9]:
datamatrix = CSVtoNumpyArray('made4.csv')
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]

In [10]:
# create a QUBO
n = 4
qubo = QuadraticProgram()
x = {(i,p): qubo.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}
qubo.minimize(quadratic={('x_{0}_{1}'.format(i,k),'x_{0}_{1}'.format(j,p)): MatrixLoc[i,j]* MatrixFlow[k,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n) })

# Constraints
for i in range(n):
    qubo.linear_constraint(linear = {'x_{0}_{1}'.format(i,p): 1 for p in range(n)},sense='==', rhs=1)
for p in range(n):
    qubo.linear_constraint(linear = {'x_{0}_{1}'.format(i,p): 1 for i in range(n)},sense='==', rhs=1)


print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 + 440 x_2_3*x_3_0 + 220 x

In [11]:
#Can only use without constraints
qp2op = QuadraticProgramToIsing()
op, offset = qp2op.encode(qubo)
print('offset: {}'.format(offset))
print('operator:')
print(op.print_details())

QiskitOptimizationError: 'An constraint exists. The method supports only model with no constraints.'

In [12]:
qaoa_mes = QAOA(quantum_instance=Aer.get_backend('qasm_simulator'))
exact_mes = NumPyMinimumEigensolver()

In [13]:
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

In [14]:
exact_result = exact.solve(qubo)
print(exact_result)

x=[0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0], fval=790.0


In [15]:
#Failed based on size
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

x=[0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0], fval=200398.0


In [16]:
rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=1, min_num_vars_optimizer=exact)

In [17]:
#Failed based on size
rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)

x=[0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0], fval=1001824.0


In [5]:
from qiskit import IBMQ

 # Load account from disk

IBMQ.save_account("3cd4df76e664d15c0a0dde90b7ce30dd1c5f0040004c5b46c8f6055623a104b2cc23d8473c156efa1dfc6c12fbb052c1c5b25a7b9f2ad26585ebb1a93f2be810", overwrite = True)
IBMQ.load_account()
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='internal', project='physics')>]

In [6]:
provider = IBMQ.get_provider(hub='ibm-q-wits', group='internal', project='physics')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_johannesburg') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_rochester') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('